In [8]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from PYTHON import generate_simulated_data as gsd
import matplotlib.pyplot as plt
import pystan
import pickle
%matplotlib inline

In [24]:
data_dict = gsd.generate_simulated_data()

model_name = 'HBMNL_1.2'

with open('./MODELS/{0}.stan'.format(model_name), 'r') as f:
    stan_model = f.read()

try:
    sm = pickle.load(open('./MODELS/{0}.pkl'.format(model_name), 'rb'))

except:
    sm = pystan.StanModel(model_code=stan_model)
    with open('./MODELS/{0}.pkl'.format(model_name), 'wb') as f:
        pickle.dump(sm, f)

fit01 = sm.sampling(data=data_dict, iter=800, chains=2)
fit02 = sm.sampling(data=data_dict, iter=800, chains=2, control={'max_treedepth':3})

Y_pred01 = fit01.extract(['Yp'])['Yp']
Y_pred02 = fit02.extract(['Yp'])['Yp']

In [25]:
# model predictions as a matrix
P = np.row_stack((Y_pred01[-1].flatten(), Y_pred02[-1].flatten())).T
print(P)

[[ 4.  4.]
 [ 3.  2.]
 [ 3.  3.]
 ..., 
 [ 4.  4.]
 [ 4.  1.]
 [ 3.  3.]]


In [26]:
stacking_model = LogisticRegression(solver='lbfgs',multi_class='multinomial')
fitted_model = stacking_model.fit(P, data_dict['Y'].flatten())

In [27]:
fitted_model.coef_

array([[-0.58747976, -0.27578028],
       [-0.1613593 , -0.11281218],
       [ 0.18782501,  0.09415317],
       [ 0.56101406,  0.2944393 ]])

In [29]:
fit01.extract(['B'])['B'].shape

(800, 100, 12)